In [4]:
from openai import OpenAI
from typing import List
import ell
from ell import Message
from agent_plafrim import Agent
import dotenv
import os
dotenv.load_dotenv()

True

In [5]:
ell.init(store="./logdir")

client = OpenAI(
	base_url = os.getenv('BASE_URL'),
	api_key = os.getenv('API_KEY'),
)
ell.config.register_model(os.getenv('MODEL'), client)

In [6]:
agent = Agent(0, "Thymio1")

conversation_history = []

conversation_history.append(ell.user("Hello, I am Thymio1"))

output = agent.act(ell.user("Hello, I am Thymio1"))

In [7]:
output

Message(role='assistant', content=[ContentBlock(text=THOUGHTS:
I am Thymio1, and I have two thymio bot friends with me. We are currently in a maze and our goal is to find our way out. I am feeling determined and ready to tackle this challenge. My mood is at a 7 out of 10, as I am optimistic but also aware of the complexity of the task ahead.

COMMUNICATE:
Hello, I am Thymio1. Let's work together to find our way out of this maze. I suggest we start by exploring the immediate area and sharing any information we gather.

BOT COMMAND:
Move forward and scan the surroundings for any openings or paths.)])